In [29]:
import requests
import numpy as np
import pandas as pd 


In [11]:
def get_geosphere_data(parameters: list = "T2M",
                       start: str = '2021-08-01T12:00', 
                       end: str = '2021-08-02T12:00', 
                       lat_lon: list = ['48.206248, 16.367569'], 
                       output_format: str = "geojson"):
    """
    function for retrieving data from geosphere API
    """
    
    base_url = 'https://dataset.api.hub.zamg.ac.at/v1/timeseries/historical/inca-v1-1h-1km'
    lat_lon_params = '&lat_lon='.join(lat_lon)
    url = f'{base_url}?parameters={parameters}&start={start}&end={end}&lat_lon={lat_lon_params}&output_format={output_format}'
    print(url)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Request failed with status code {response.status_code}.')

In [12]:
data = get_geosphere_data()

https://dataset.api.hub.zamg.ac.at/v1/timeseries/historical/inca-v1-1h-1km?parameters=T2M&start=2021-08-01T12:00&end=2021-08-02T12:00&lat_lon=48.206248, 16.367569&output_format=geojson


In [30]:
np.mean(data["features"][0]["properties"]["parameters"]["T2M"]["data"])

18.7552

In [ ]:
# function
# input bounding box and get data (temp, prec, rhum, wind) for last 24 hours of 
